In [1]:
# This is my personal data folder. Change to your own
dir = "/Users/piromast/Documents/Kaggle/SantanderData/"

In [2]:
# Import necessary stuff
from pandas import *
import os.path

In [3]:
# Import data (first 1000 rows for now)
data=read_csv(os.path.join(dir,'train_ver2.csv'),nrows=1000)

In [48]:
# Import description table (describing the meaning of each column)
description = read_csv(os.path.join(dir,'column_description.csv'),encoding = 'latin1')
description

,Column name,Description
0,fecha_dato,The table is partitioned for this column
1,ncodpers,Customer code
2,ind_empleado,"Employee index: A active, B ex employed, F fil..."
3,pais_residencia,Customer's Country residence
4,sexo,Customer's sex
5,age,Age
6,fecha_alta,The date in which the customer became as the f...
7,ind_nuevo,New customer Index. 1 if the customer register...
8,antiguedad,Customer seniority (in months)
9,indrel,"1 (First/Primary), 99 (Primary customer during..."


In [30]:
# Select product column names and build indeces range
column_names = data.columns.values
product_names = [(i, s) for i, s in enumerate(column_names) if 'ind_' in s and '_ult1' in s]
product_indeces = [e[0] for e in product_names]
mInd = min(product_indeces)
MInd = max(product_indeces)
product_indeces_range = range(mInd,MInd)

In [31]:
product_names

[(24, 'ind_ahor_fin_ult1'),
 (25, 'ind_aval_fin_ult1'),
 (26, 'ind_cco_fin_ult1'),
 (27, 'ind_cder_fin_ult1'),
 (28, 'ind_cno_fin_ult1'),
 (29, 'ind_ctju_fin_ult1'),
 (30, 'ind_ctma_fin_ult1'),
 (31, 'ind_ctop_fin_ult1'),
 (32, 'ind_ctpp_fin_ult1'),
 (33, 'ind_deco_fin_ult1'),
 (34, 'ind_deme_fin_ult1'),
 (35, 'ind_dela_fin_ult1'),
 (36, 'ind_ecue_fin_ult1'),
 (37, 'ind_fond_fin_ult1'),
 (38, 'ind_hip_fin_ult1'),
 (39, 'ind_plan_fin_ult1'),
 (40, 'ind_pres_fin_ult1'),
 (41, 'ind_reca_fin_ult1'),
 (42, 'ind_tjcr_fin_ult1'),
 (43, 'ind_valo_fin_ult1'),
 (44, 'ind_viv_fin_ult1'),
 (45, 'ind_nomina_ult1'),
 (46, 'ind_nom_pens_ult1'),
 (47, 'ind_recibo_ult1')]

In [37]:
# Build a dataframe only with products
product_df = data.iloc[:,product_indeces_range]

In [75]:
# Build a Series with number of products and use description as indeces
desclist=description.iloc[product_indeces_range,1].tolist()
num_products = Series(product_df.sum().tolist(),index=desclist)
num_products


Saving Account               0
Guarantees                   0
Current Accounts           928
Derivada Account             0
Payroll Account             34
Junior Account               8
Ms particular Account       1
particular Account           0
particular Plus Account      0
Short-term deposits          1
Medium-term deposits         0
Long-term deposits           6
e-account                   74
Funds                        1
Mortgage                     0
Pensions                     0
Loans                        1
Taxes                       10
Credit Card                 11
Securities                   5
Home Account                 0
Payroll                     19
Pensions                    21
dtype: int64